#### To access the historical stock data, we'll be using yfinance which pulls data from Yahoo! Finance

In [ ]:
!pip install yfinance

#### Importing necessary packages

In [1]:
import pandas as pd
import datetime
import numpy as np

import yfinance as yf
from prettytable import PrettyTable 

#### Setting the capital and duration

In [2]:
# Amount of capital to both Amazon and Apple
principal_amount = 1000
# In DD/MM/YYYY format
start_date = '2020-07-01'
end_date = '2021-09-01'

#### Pulling the stock data

In [3]:
# Using yahoo finance api to access stock data for inputted time period
amzn_data = yf.download('AMZN', start=start_date, end=end_date)
aapl_data = yf.download('AAPL', start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


#### Checking how the data looks like

In [4]:
amzn_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-06-30,2685.070068,2769.629883,2675.030029,2758.820068,2758.820068,3769700
2020-07-01,2757.989990,2895.000000,2754.000000,2878.699951,2878.699951,6363400
2020-07-02,2912.010010,2955.560059,2871.100098,2890.300049,2890.300049,6593400
2020-07-06,2934.969971,3059.879883,2930.000000,3057.040039,3057.040039,6880600
2020-07-07,3058.550049,3069.550049,2990.000000,3000.120117,3000.120117,5257500


In [5]:
aapl_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-06-30,90.019997,91.495003,90.000000,91.199997,90.454079,140223200
2020-07-01,91.279999,91.839996,90.977501,91.027496,90.282982,110737200
2020-07-02,91.962502,92.617500,90.910004,91.027496,90.282982,114041600
2020-07-06,92.500000,93.945000,92.467499,93.462502,92.698074,118655600
2020-07-07,93.852501,94.654999,93.057503,93.172501,92.410439,112424400


#### Removing columns which aren't necessary 

In [6]:
# Cleaning the data
amzn_data.drop(['High','Low','Adj Close', 'Volume'],axis=1,inplace=True)
aapl_data.drop(['High','Low','Adj Close', 'Volume'],axis=1,inplace=True)

In [7]:
amzn_data.head()

,Open,Close
Date,,
2020-06-30,2685.070068,2758.820068
2020-07-01,2757.989990,2878.699951
2020-07-02,2912.010010,2890.300049
2020-07-06,2934.969971,3057.040039
2020-07-07,3058.550049,3000.120117


In [8]:
aapl_data.head()

,Open,Close
Date,,
2020-06-30,90.019997,91.199997
2020-07-01,91.279999,91.027496
2020-07-02,91.962502,91.027496
2020-07-06,92.500000,93.462502
2020-07-07,93.852501,93.172501


#### Calculating the overnight return

Formula for overnight return = ((Open - Previous Close) / Close) * 100  
The first transaction would be buying on the start date, so we won't consider the start date's open price, which represents the date of selling.   
Similarly, the last transaction would be selling on the end date, so we won't consider the end date's close price which represents the date of buying.   

In [9]:
# (Open - Previous Close)
overnight_return = amzn_data['Open'].to_numpy()[1:] - amzn_data['Close'].to_numpy()[:-1]
# Dividing by Previous Close
overnight_return = overnight_return / amzn_data['Close'].to_numpy()[:-1]
# Converting to percentile by multiplying 100
overnight_return = overnight_return * 100

# We start the buying process at close price at first day, the first day's return would be zero
overnight_return = np.insert(overnight_return, 0, 0.0)

amzn_data['Overnight Return %'] = overnight_return

In [10]:
amzn_data.head()

,Open,Close,Overnight Return %
Date,,,
2020-06-30,2685.070068,2758.820068,0.000000
2020-07-01,2757.989990,2878.699951,-0.030088
2020-07-02,2912.010010,2890.300049,1.157122
2020-07-06,2934.969971,3057.040039,1.545512
2020-07-07,3058.550049,3000.120117,0.049395


#### Repeating the process on AAPL stock

In [11]:
# (Open - Previous Close)
overnight_return = aapl_data['Open'].to_numpy()[1:] - aapl_data['Close'].to_numpy()[:-1]
# Dividing by Previous Close
overnight_return = overnight_return / aapl_data['Close'].to_numpy()[:-1]
# Converting to percentile by multiplying 100
overnight_return = overnight_return * 100

# We start the buying process at close price at first day, the first day's return would be zero
overnight_return = np.insert(overnight_return, 0, 0.0)

aapl_data['Overnight Return %'] = overnight_return

In [12]:
aapl_data.head()

,Open,Close,Overnight Return %
Date,,,
2020-06-30,90.019997,91.199997,0.000000
2020-07-01,91.279999,91.027496,0.087721
2020-07-02,91.962502,91.027496,1.027168
2020-07-06,92.500000,93.462502,1.617647
2020-07-07,93.852501,93.172501,0.417279


#### Testing

In [13]:
# Finding the total PnL from day-night stratergy
print('$',str(principal_amount), ' invested in Amazon Stock on',start_date)
amzn_profit = 0
for i, date in enumerate(amzn_data.index[1:]):
    # Normalizing the Timestamp to DD/MM/YYYY
    date = date.strftime("%d/%m/%Y")
    amzn_pnl =  (amzn_data['Overnight Return %'][i+1]/100) * principal_amount
    amzn_profit = amzn_profit + amzn_pnl
print('Investment value after Day-night on ',end_date,':',(principal_amount + amzn_profit).round(1))
print('Day-night PnL : ',((amzn_profit/principal_amount) * 100).round(2),'%')

# Finding the total PnL from simply holding (assuming fractional shares)
buy_price = amzn_data.iloc[0][0]
sell_price = amzn_data.iloc[-1][0]
amzn_profit = ((principal_amount/buy_price) * sell_price) - principal_amount
print('Investment value after holding on ',end_date,':',(principal_amount + amzn_profit).round(1))
print('Holding PnL : ',((amzn_profit/principal_amount) * 100).round(2),'%')

$ 1000  invested in Amazon Stock on 2020-07-01
Investment value after Day-night on  2021-09-01 : 1525.9
Day-night PnL :  52.59 %
Investment value after holding on  2021-09-01 : 1275.5
Holding PnL :  27.55 %


In [14]:
# Printing the total PnL
print('$',str(principal_amount), ' invested in Apple Stock on',start_date)
aapl_profit = 0
for i, date in enumerate(aapl_data.index[1:]):
    # Normalizing the Timestamp to DD/MM/YYYY
    date = date.strftime("%d/%m/%Y")
    aapl_pnl =  (aapl_data['Overnight Return %'][i+1] / 100) * principal_amount
    aapl_profit = aapl_profit + aapl_pnl
print('Investment value after Day-night on ',end_date,':',(principal_amount + aapl_profit).round(1))
print('Day-night PnL :: ',((aapl_profit/principal_amount) * 100).round(2),'%')

# Finding the total PnL from simply holding (assuming fractional shares)
buy_price = aapl_data.iloc[0][0]
sell_price = aapl_data.iloc[-1][0]
aapl_profit = (principal_amount/buy_price) * sell_price - principal_amount
print('Investment value after holding on ',end_date,':',(principal_amount + aapl_profit).round(1))
print('Holding PnL : ',((aapl_profit/principal_amount) * 100).round(2),'%')

$ 1000  invested in Apple Stock on 2020-07-01
Investment value after Day-night on  2021-09-01 : 1538.1
Day-night PnL ::  53.81 %
Investment value after holding on  2021-09-01 : 1695.8
Holding PnL :  69.58 %


### Visualing on different timeframes

In [15]:
table = PrettyTable(["Duration", "Stock", "Holding", "Day-night Strategy"]) 

In [16]:
end_date = '2021-09-01'
duration_and_start_date = [\
                        ['1 week', '2021-08-25'], ['1 month', '2021-08-01'],
                        ['1 quarter', '2021-06-01'], ['1 year', '2020-09-01'],
                        ['1 decade', '2011-09-01']
                        ]

In [17]:
for duration, start_date in duration_and_start_date:
    # Using yahoo finance api to access stock data for inputted time period
    amzn_data = yf.download('AMZN', start=start_date, end=end_date)
    aapl_data = yf.download('AAPL', start=start_date, end=end_date)

    # Cleaning the data
    amzn_data.drop(['High','Low','Adj Close', 'Volume'],axis=1,inplace=True)
    aapl_data.drop(['High','Low','Adj Close', 'Volume'],axis=1,inplace=True)

    # Calculating the overnight return for AMZN
    overnight_return = amzn_data['Open'].to_numpy()[1:] - amzn_data['Close'].to_numpy()[:-1]
    overnight_return = overnight_return / amzn_data['Close'].to_numpy()[:-1]
    overnight_return = overnight_return * 100
    overnight_return = np.insert(overnight_return, 0, 0.0)
    amzn_data['Overnight Return %'] = overnight_return

    # Calculating the overnight return for AAPL
    overnight_return = aapl_data['Open'].to_numpy()[1:] - aapl_data['Close'].to_numpy()[:-1]
    overnight_return = overnight_return / aapl_data['Close'].to_numpy()[:-1]
    overnight_return = overnight_return * 100
    overnight_return = np.insert(overnight_return, 0, 0.0)
    aapl_data['Overnight Return %'] = overnight_return

    # Finding the PnLs for AMZN (assuming fractional shares)
    amzn_profit = 0
    for i, date in enumerate(amzn_data.index[1:]):
        # Normalizing the Timestamp to DD/MM/YYYY
        date = date.strftime("%d/%m/%Y")
        amzn_pnl =  (amzn_data['Overnight Return %'][i+1] / 100) * principal_amount
        amzn_profit = amzn_profit + amzn_pnl
    dn_pnl = str(((amzn_profit/principal_amount)*100).round(2)) + '%'
    buy_price = amzn_data.iloc[0][0]
    sell_price = amzn_data.iloc[-1][0]
    amzn_profit = (principal_amount/buy_price) * sell_price - principal_amount
    hold_pnl = str(((amzn_profit/principal_amount)*100).round(2)) + '%'
    # Adding the data to the table
    table.add_row([duration, 'AMZN', hold_pnl, dn_pnl])

    # Finding the PnLs for AAPL (assuming fractional shares)
    aapl_profit = 0
    for i, date in enumerate(aapl_data.index[1:]):
        # Normalizing the Timestamp to DD/MM/YYYY
        date = date.strftime("%d/%m/%Y")
        aapl_pnl =  (aapl_data['Overnight Return %'][i+1] / 100) * principal_amount
        aapl_profit = aapl_profit + aapl_pnl
    dn_pnl = str(((aapl_profit/principal_amount)*100).round(2)) + '%'
    buy_price = aapl_data.iloc[0][0]
    sell_price = aapl_data.iloc[-1][0]
    aapl_profit = (principal_amount/buy_price) * sell_price - principal_amount
    hold_pnl = str(((aapl_profit/principal_amount)*100).round(2)) + '%'
    # Adding the data to the table
    table.add_row([duration, 'AAPL', hold_pnl, dn_pnl])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [18]:
print(table)

+-----------+-------+----------+--------------------+
|  Duration | Stock | Holding  | Day-night Strategy |
+-----------+-------+----------+--------------------+
|   1 week  |  AMZN |  4.41%   |       0.97%        |
|   1 week  |  AAPL |  2.15%   |       0.05%        |
|  1 month  |  AMZN |  2.14%   |       2.13%        |
|  1 month  |  AAPL |   4.3%   |       -1.13%       |
| 1 quarter |  AMZN |  5.59%   |       -1.28%       |
| 1 quarter |  AAPL |  22.05%  |       -1.56%       |
|   1 year  |  AMZN |  0.46%   |       35.43%       |
|   1 year  |  AAPL |  19.66%  |       27.99%       |
|  1 decade |  AMZN | 1513.42% |      308.51%       |
|  1 decade |  AAPL | 994.42%  |       207.1%       |
+-----------+-------+----------+--------------------+
